#### Each panel has some dates missing: add those dates, for all hours

In [8]:
import pandas as pd
from config import getClient
from checkMissing import checkMissing
from datetime import datetime, timedelta
import time

In [9]:
datalib = getClient("OutFront")

In [10]:
df = pd.read_csv(datalib+"1fillRateFilter.csv")

# Convert their date format to a standard date
df["date"] = pd.to_datetime(df["date"])

In [11]:
# Use the population and hours from the week prior, or following,
# for this record
def getVals(df, d):    
    try:
        dt = d["date"] - timedelta(days=7)
        tmp = df.loc[d["panel"], dt]
    except:
        dt  = d["date"] + timedelta(days=7)        
        tmp = df.loc[d["panel"], dt]
    pop   = tmp["population"].reset_index(drop=True)
    hours = tmp["hour"].reset_index(drop=True)
    return pop, hours

In [12]:
# A panel is missing dates
def addDates(panel, missing):
    dfList  = []
    d = {}
    d["panel"] = panel
    
    for dt in missing:
        d["date"]  = dt
        d["population"], d["hour"] = getVals(df, d)
        imputed = pd.DataFrame.from_dict(d)
        imputed = imputed.set_index(["panel", "date"])
        dfList.append(imputed)
    return dfList

In [ ]:
popBefore = df["population"].sum()

# Check each panel for missing dates
df  = df.set_index("panel")
grp = df.groupby(level=0)
df  = df.reset_index()
df  = df.set_index(["panel", "date"])

start = time.time()
count = duration = 0
for idx, val in grp:
    count += 1
    panel = idx
    missing = checkMissing(val, "day")
    if len(missing) > 0:
        dfList = addDates(panel, missing)
        for x in dfList:
            df = df.append(x)

duration = time.time() - start
print("{:,.0f} panels processed in {:.2f} minutes".format(count, (duration)/60))

/home/tbrownex/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  
/home/tbrownex/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  if __name__ == '__main__':


In [ ]:
df = df.reset_index()
df = df.set_index(["panel", "date", "hour"])
df = df.sort_index()
df.to_csv(datalib+"2a_dates.csv")